In [1]:
import os
GPU_ID = "0"

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=GPU_ID

In [3]:
import sys
sys.path.append("/home/ailie/Repos/BBAttacks/attacks/")
sys.path.append("/home/ailie/Repos/BBAttacks/utils/")

import utils
from data_manager import load_data

# Black Box Attacks
import random_noise
import EvoStrategy
import SimbaWrapper

In [4]:
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
from tensorflow.keras.datasets import cifar100
from tensorflow.keras import backend as K
from tensorflow import keras as keras
importlib.reload(EvoStrategy)
import json
import time
import datetime
from pathlib import Path
from tqdm.notebook import tqdm
import sys
import gc

In [5]:
import tensorflow as tf
DEVICE = f'/GPU:{GPU_ID}'

In [6]:
CIFAR100_VGG_PATH = "/home/ailie/Repos/BBAttacks/models/cifar100vgg/"
sys.path.append(CIFAR100_VGG_PATH)
import cifar100vgg

In [7]:
importlib.reload(cifar100vgg)
WEIGHTS_FILE_NAME = "cifar100vgg.h5"
model = cifar100vgg.cifar100vgg(train=False, weights_path=CIFAR100_VGG_PATH + WEIGHTS_FILE_NAME)

In [8]:
NUM_CLASSES = 100

In [9]:
LOAD_DATA = True
SAVE_DATA = False

In [10]:
if LOAD_DATA:
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

    y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
    y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)
    
    if SAVE_DATA:
        np.save("x_test.npy", x_test)
        np.save("y_test.npy", y_test)

In [11]:
x_train = x_train.astype('int')
x_test = x_test.astype('int')

In [12]:
gc.collect()

119

<h4> Check the training accuracy </h4>

In [13]:
with tf.device(DEVICE):
    preds_train = model.predict(x_train)
    
preds_train_labels = np.argmax(preds_train, axis=1)

In [14]:
true_train_labels = np.argmax(y_train, axis=1)

In [15]:
acc_train = np.mean(true_train_labels == preds_train_labels)
print(f"Training accuracy: {acc_train}")

Training accuracy: 0.99638


<h4> Check the test accuracy </h4>

In [16]:
with tf.device(DEVICE):
    preds_test = model.predict(x_test)

preds_test_labels = np.argmax(preds_test, axis=1)

In [17]:
true_test_labels = np.argmax(y_test, axis=1)

In [18]:
acc_test = np.mean(true_test_labels == preds_test_labels)
print(f"Test accuracy: {acc_test}")

Test accuracy: 0.7048


<h4> Save the images classified correctly by the network </h4>

In [19]:
x_test_correct = x_test[preds_test_labels == true_test_labels]
y_test_correct = y_test[preds_test_labels == true_test_labels]

<h2> Run baseline EvoBA </h2>

In [20]:
importlib.reload(EvoStrategy)
importlib.reload(utils)

<module 'utils' from '/home/ailie/Repos/BBAttacks/utils/utils.py'>

In [21]:
EXPERIMENT_NAME_MLFLOW = "CIFAR100"

In [22]:
from utils import AttackType

In [23]:
GENERATION_SIZE = 30
PIXEL_COUNT = 1
STEPS = 80

In [24]:
LEFT_IDX_ATTACK_IMAGES = 0
RIGHT_IDX_ATTACK_IMAGES = 1000
SAMPLE_IMAGES = x_test_correct[LEFT_IDX_ATTACK_IMAGES: RIGHT_IDX_ATTACK_IMAGES]
SAMPLE_Y = y_test_correct[LEFT_IDX_ATTACK_IMAGES: RIGHT_IDX_ATTACK_IMAGES]

In [25]:
VERBOSE = False

In [26]:
%%time

with tf.device(DEVICE):
    adv_evo_strategy = {}
    
    for index in tqdm(range(len(SAMPLE_IMAGES))):
        img = SAMPLE_IMAGES[index]
        label = SAMPLE_Y[index]
        
        true_label = np.argmax(label)

        adv_evo_strategy[index] = EvoStrategy.AdversarialPerturbationEvoStraegy(
            model=model,
            img=img,
            label=true_label,
            generation_size=GENERATION_SIZE, 
            one_step_perturbation_pixel_count=PIXEL_COUNT,
            verbose=VERBOSE,
            zero_one_scale=False
        )

        no_steps = adv_evo_strategy[index].run_adversarial_attack(steps=STEPS) 

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 6min 23s, sys: 8.33 s, total: 6min 31s
Wall time: 6min 10s


In [27]:
evoba_stats = utils.get_evoba_stats(adv_evo_strategy)

In [28]:
utils.print_evoba_stats(evoba_stats)


EvoBA STATS (L0 attack)
____________________
Perturbed successfully 1000/1000 images
Average query count: 191.08
Average l0 distance: 18.871

Median query count: 151.0
Median l0 dist: 15.0

Max query count: 1591
Max l0 dist: 150
____________________



In [29]:
utils.generate_mlflow_logs(
    strategy_objects=adv_evo_strategy, 
    attack_type=AttackType.EVOBA, 
    unperturbed_images=SAMPLE_IMAGES, 
    run_name="EVOBA", 
    experiment_name=EXPERIMENT_NAME_MLFLOW
)

Ended previous run
Logging run EVOBA under experiment CIFAR100


'SUCCESS'

<Figure size 432x288 with 0 Axes>

<h2> Run Epsilon Greedy </h2>

In [30]:
import EpsilonGreedyAttack
importlib.reload(EpsilonGreedyAttack)

<module 'EpsilonGreedyAttack' from '/home/ailie/Repos/BBAttacks/attacks/EpsilonGreedyAttack.py'>

In [31]:
# Generate grid of areas (one area = one bandit)
BATCH_LEN = 4
IMAGE_SIZE = 32
assert IMAGE_SIZE % BATCH_LEN == 0

pixel_groups = []
for i in range(int(IMAGE_SIZE/BATCH_LEN)):
    for j in range(int(IMAGE_SIZE/BATCH_LEN)):
        current_group = []
        for pixel_i in range(BATCH_LEN):
            for pixel_j in range(BATCH_LEN):
                pixel = (BATCH_LEN * i + pixel_i, BATCH_LEN * j + pixel_j)
                current_group.append(pixel)
        pixel_groups.append(current_group)

In [ ]:
%%time

EPS = 0.1

with tf.device(DEVICE):
    adv_evo_strategy_epsilon = {}
    
    for index in tqdm(range(len(SAMPLE_IMAGES))):
        img = SAMPLE_IMAGES[index]
        label = SAMPLE_Y[index]
        
        true_label = np.argmax(label)

        adv_evo_strategy_epsilon[index] = EpsilonGreedyAttack.EpsilonGreedyAttack(
            model=model,
            img=img,
            one_hot_label=label,
            pixel_groups=pixel_groups,
            epsilon=EPS
            # generation_size=GENERATION_SIZE, 
            # one_step_perturbation_pixel_count=PIXEL_COUNT,
            # verbose=VERBOSE,
            # zero_one_scale=False
        )

        no_steps = adv_evo_strategy_epsilon[index].run_attack()

params_eps_greedy = {
    "epsilon": EPS
}

  0%|          | 0/1000 [00:00<?, ?it/s]

In [ ]:
eps_greedy_stats = utils.get_epsgreedy_stats(adv_evo_strategy_epsilon, SAMPLE_IMAGES)

In [ ]:
utils.print_evoba_stats(eps_greedy_stats)

In [ ]:
utils.generate_mlflow_logs(
    strategy_objects=adv_evo_strategy_epsilon, 
    attack_type=AttackType.EPSGREEDY, 
    unperturbed_images=SAMPLE_IMAGES, 
    run_name="EPSGREEDY", 
    experiment_name=EXPERIMENT_NAME_MLFLOW,
    additional_params=params_eps_greedy
)

In [ ]:
importlib.reload(utils)
from utils import AttackType

<h2> Run Epsilon Greedy Threshold </h2>

In [ ]:
import EpsilonGreedyAttackThreshold
importlib.reload(EpsilonGreedyAttackThreshold)

In [ ]:
# Generate grid of areas (one area = one bandit)
BATCH_LEN = 4
IMAGE_SIZE = 32
assert IMAGE_SIZE % BATCH_LEN == 0

pixel_groups = []
for i in range(int(IMAGE_SIZE/BATCH_LEN)):
    for j in range(int(IMAGE_SIZE/BATCH_LEN)):
        current_group = []
        for pixel_i in range(BATCH_LEN):
            for pixel_j in range(BATCH_LEN):
                pixel = (BATCH_LEN * i + pixel_i, BATCH_LEN * j + pixel_j)
                current_group.append(pixel)
        pixel_groups.append(current_group)

In [ ]:
%%time

EPS = 0.1
THRESHOLD = 0.025
INCREASE_FACTOR = 1.05
DECAY_FACTOR = 0.5
MAX_ROUNDS_UNTIL_DECAY = 20

with tf.device(DEVICE):
    adv_evo_strategy_eps_threshold = {}
    
    for index in tqdm(range(len(SAMPLE_IMAGES))):
        img = SAMPLE_IMAGES[index]
        label = SAMPLE_Y[index]
        
        true_label = np.argmax(label)

        adv_evo_strategy_eps_threshold[index] = EpsilonGreedyAttackThreshold.EpsilonGreedyAttackThreshold(
            model=model,
            img=img,
            one_hot_label=label,
            pixel_groups=pixel_groups,
            threshold=THRESHOLD,
            increase_factor=INCREASE_FACTOR,
            decay_factor=DECAY_FACTOR,
            epsilon=EPS,
            max_rounds_until_decay=MAX_ROUNDS_UNTIL_DECAY
            # generation_size=GENERATION_SIZE, 
            # one_step_perturbation_pixel_count=PIXEL_COUNT,
            # verbose=VERBOSE,
            # zero_one_scale=False
        )

        no_steps = adv_evo_strategy_eps_threshold[index].run_attack()

params_eps_threshold = {
    "epsilon": EPS,
    "increase_fact": INCREASE_FACTOR,
    "decay_fact": DECAY_FACTOR,
    "rounds_decay": MAX_ROUNDS_UNTIL_DECAY
}

In [ ]:
eps_threshold_stats = utils.get_epsgreedy_stats(adv_evo_strategy_eps_threshold, SAMPLE_IMAGES)

In [ ]:
utils.print_evoba_stats(eps_threshold_stats)

In [ ]:
utils.generate_mlflow_logs(
    strategy_objects=adv_evo_strategy_eps_threshold, 
    attack_type=AttackType.EPSGREEDY, 
    unperturbed_images=SAMPLE_IMAGES, 
    run_name="EPSGREEDY_THRESHOLD", 
    experiment_name=EXPERIMENT_NAME_MLFLOW,
    additional_params=params_eps_threshold
)